# Single Link

## 1.Preparação do ambiente

In [1]:
# Importando bibliotecas utilizadas
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from math import sqrt, pow
from datetime import datetime
from time import sleep

%matplotlib inline

## 2.Algoritmo

### 2.2.Implementação do Algoritmo

In [4]:
class Distancia:
    def __init__(self, distancia, objetos):
        self.distancia = distancia
        self.objetos = objetos
        
    def __str__(self):
        return "{}, {}".format(self.distancia, self.objetos)
    
    def __repr__(self):
        return repr((self.distancia, self.objetos))

In [5]:
#Distancia Euclidiana
def distancia_euclidiana(p1, p2):
    distancia = sqrt(pow((p1[0] - p2[0]), 2) + pow((p1[1] - p2[1]), 2))
    return distancia

In [6]:
# Distancia com o método sorted
def calcular_distancias(d1, d2, numero_de_objetos):
    lista = []
    for i in range(numero_de_objetos):
        for j in range(i+1, numero_de_objetos):
            distancia = distancia_euclidiana([d1[i], d2[i]], [d1[j], d2[j]])
            objetos = [i, j]
            
            d = Distancia(distancia, objetos)
            lista.append(d)

    nova_lista = sorted(lista, key= lambda distancia: distancia.distancia)        
    return nova_lista


In [7]:
def iniciliza_clusters(numero_de_objetos):
    clusters = []
    for i in range(numero_de_objetos):
        clusters.append(i)

    return clusters    

In [8]:
def agrupa_clusters(clusters, clusters_mais_proximos):
    obj1, obj2 = clusters_mais_proximos
    cluster1 = clusters[obj1]
    cluster2 = clusters[obj2]
    
    if cluster1 == cluster2:
        return False
    
    if cluster1 < cluster2:
        menor = cluster1
        maior = cluster2
    else:
        menor = cluster2
        maior = cluster1
        
    for i in range(len(clusters)):
        if clusters[i] == maior:
            clusters[i] = menor
    
    return True

In [15]:
def ajusta_indice(clusters):
    clusters = pd.Series(clusters)
    unicos = clusters.unique()
    hash_map = {}
    
    for i in range(len(unicos)):
        hash_map[unicos[i]] = i
    
    for i in range(len(clusters)):
        clusters[i] = hash_map[clusters[i]]
    

In [27]:
def salvar_arquivo(dataset, clusters, nome, i):
    ajusta_indice(clusters)
    dataset['clusters'] = clusters
    
    dataset.drop(axis='columns', labels=['d1', 'd2']).to_csv(f'datasets/{nome}_{i:03d}.csv')

In [28]:
def single_link(dataset, k_min, k_max, nome):
    numero_de_objetos = len(dataset)
    clusters = iniciliza_clusters(numero_de_objetos)
    fila_com_distancias = calcular_distancias(dataset['d1'], dataset['d2'], numero_de_objetos)

    
    i = numero_de_objetos
    while i > k_min:
        objetos = fila_com_distancias.pop(0)
        
        if agrupa_clusters(clusters, objetos.objetos):
            i -= 1
            if i <= k_max:
                salvar_arquivo(dataset, clusters, nome, i)
    return clusters